# YOLOv8 Parking Slot Detection Model Training

This notebook trains a YOLOv8 model to detect free and full parking slots using the Autonomous Parking System dataset.

## Install Dependencies

Install required libraries for YOLOv8 training.

In [ ]:
import subprocess
import sys

# Install ultralytics (YOLOv8)
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "ultralytics"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "torch"])

print("✓ Dependencies installed successfully!")

## Import Libraries

In [ ]:
from ultralytics import YOLO
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

print("✓ Libraries imported successfully!")

## Dataset Configuration

Set up the dataset path and verify the structure.

In [ ]:
# Dataset path
dataset_path = Path("Autonomous parking system.v2i.yolov8")
data_yaml = dataset_path / "data.yaml"

# Verify dataset exists
if dataset_path.exists():
    print(f"✓ Dataset found at: {dataset_path.absolute()}")
    
    # List dataset structure
    print("\nDataset Structure:")
    for split in ['train', 'valid', 'test']:
        split_path = dataset_path / split
        if split_path.exists():
            images = list((split_path / 'images').glob('*'))
            labels = list((split_path / 'labels').glob('*'))
            print(f"  {split}: {len(images)} images, {len(labels)} labels")
else:
    print(f"✗ Dataset not found at: {dataset_path}")

# Check if data.yaml exists
if data_yaml.exists():
    print(f"\n✓ data.yaml found")
    with open(data_yaml, 'r') as f:
        print("data.yaml content:")
        print(f.read())
else:
    print(f"✗ data.yaml not found at: {data_yaml}")

## Train YOLOv8 Model

Train a YOLOv8 model on the parking slot detection dataset.

In [ ]:
# Load a pretrained YOLOv8 model
print("Loading pretrained YOLOv8 model...")
model = YOLO("yolov8m.pt")  # medium model - balanced between speed and accuracy
print("✓ Model loaded successfully!")

# Train the model
print("\nStarting training...")
print("This may take several minutes depending on your hardware...\n")

results = model.train(
    data=str(data_yaml),           # path to data.yaml
    epochs=50,                      # number of epochs
    imgsz=640,                      # image size
    batch=8,                        # batch size (adjust based on GPU memory)
    patience=10,                    # early stopping patience
    device=0,                       # GPU device (0 for first GPU, CPU if no GPU)
    project="parking_detection",    # project directory
    name="yolov8m_parking",         # experiment name
    exist_ok=False,                 # don't overwrite existing experiment
    save=True,                      # save training checkpoints
    save_period=10,                 # save every N epochs
    verbose=True,                   # verbose output
    pretrained=True,                # use pretrained weights
    optimizer="SGD",                # optimizer
    seed=42                         # for reproducibility
)

print("\n✓ Training completed!")
print(f"Results saved to: parking_detection/yolov8m_parking")

## Validate Model Performance

Evaluate the trained model on the validation set.

In [ ]:
# Load the best trained model
best_model_path = Path("parking_detection/yolov8m_parking/weights/best.pt")

if best_model_path.exists():
    print(f"Loading best model from: {best_model_path}")
    best_model = YOLO(str(best_model_path))
    
    # Validate on the validation set
    print("\nValidating on validation set...")
    val_results = best_model.val()
    
    print("\n" + "="*50)
    print("VALIDATION METRICS")
    print("="*50)
    print(f"mAP50: {val_results.box.map50:.4f}")
    print(f"mAP50-95: {val_results.box.map:.4f}")
    print(f"Precision: {val_results.box.mp:.4f}")
    print(f"Recall: {val_results.box.mr:.4f}")
else:
    print(f"Best model not found at: {best_model_path}")
    print("Make sure training completed successfully.")

## Test on Validation Images

Run inference on some validation images to visualize results.

In [ ]:
from pathlib import Path
import os

# Get validation images
valid_images_path = Path("Autonomous parking system.v2i.yolov8/valid/images")

if valid_images_path.exists():
    image_files = list(valid_images_path.glob("*.jpg")) + list(valid_images_path.glob("*.png"))
    
    if image_files:
        # Run inference on first 5 images
        print(f"Running inference on {min(5, len(image_files))} validation images...\n")
        
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        axes = axes.flatten()
        
        for idx, img_path in enumerate(image_files[:6]):
            if idx >= 6:
                break
                
            print(f"Processing: {img_path.name}")
            
            # Run inference
            results = best_model.predict(source=str(img_path), conf=0.5, verbose=False)
            result = results[0]
            
            # Get annotated image
            annotated_img = result.plot()
            
            # Display
            axes[idx].imshow(cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB))
            axes[idx].set_title(f"Detections: {len(result.boxes)}")
            axes[idx].axis("off")
        
        plt.tight_layout()
        plt.savefig("parking_detection/sample_predictions.png", dpi=100, bbox_inches='tight')
        plt.show()
        
        print("\n✓ Sample predictions saved to: parking_detection/sample_predictions.png")
    else:
        print("No validation images found!")
else:
    print(f"Validation images directory not found: {valid_images_path}")

## Export and Use the Model

Export the trained model to different formats and prepare for deployment.

In [ ]:
# Export model to different formats
print("Exporting model to different formats...\n")

# Export to PyTorch (default)
print("1. PyTorch format (.pt):")
print(f"   Location: {best_model_path}")

# Export to ONNX (for cross-platform compatibility)
print("\n2. ONNX format (.onnx):")
onnx_path = best_model.export(format='onnx', dynamic=True)
print(f"   Location: {onnx_path}")

# Export to TorchScript
print("\n3. TorchScript format (.torchscript):")
torchscript_path = best_model.export(format='torchscript')
print(f"   Location: {torchscript_path}")

print("\n" + "="*50)
print("TRAINING SUMMARY")
print("="*50)
print(f"✓ Model successfully trained!")
print(f"✓ Best model saved: parking_detection/yolov8m_parking/weights/best.pt")
print(f"✓ Training logs: parking_detection/yolov8m_parking/")
print(f"\nUSEDFOR INFERENCE:")
print(f"  from ultralytics import YOLO")
print(f"  model = YOLO('parking_detection/yolov8m_parking/weights/best.pt')")
print(f"  results = model.predict(source='image.jpg')")
print("="*50)